At a very high level, MCMC tries to sample from an unknown target distribution $p(x)$ using a known function $f(x)$ where 
$$ p(x) = \frac{f(x)}{Z}$$
This is done in a Markov chain. One way is using the Metropolis Hastings algorithm. Essentially the algorithm samples the samples $x$ from a distribution $g(x)$ eg $g(x) = \mathcal{N}(x|\mu,\sigma)$, then samples the next sample depending on the previous sample $g(x_{t+1}|x_t)$. The sample generated in this step is accepted with probability of acceptance $A(x_t \rightarrow x_{t+1})$. 

If we can show that the probablity of going from any one state $x_t$ to another state $x_{t+1}$ follows the detail balance condition then we know that we have reached a stationary condition, and the Markov chain will be sampling from $p(x)$ for the next steps. The detail balance equation is simple $p(a) T(a \rightarrow b)=p(b) T(b \rightarrow q)$ where $T(a\rightarrow b)$ is the transition probablity for going from state a to state b. Since we don't know $p(a)$, we know $f(a)\approx p(a)$ with some proposed distribution $g(a|b)$ (eg a gaussian) to go from state $a$ to state $b$. 

$$\frac{f(a)}{Z} g(b / a) A(a \rightarrow b)=\frac{f(b)}{Z} g(a \mid b) A(b \rightarrow a)$$

From above we have 
$$\frac{A(a \rightarrow b)}{A(b \rightarrow a)}=\frac{f(b)}{f(a)} \times \frac{g(a \mid b)}{g(b \mid a)}$$

If $\frac{f(b)}{f(a)} \times \frac{g(a \mid b)}{g(b \mid a)} \lt 1$:

$A(a \rightarrow b)=\frac{f(b)}{f(a)} \times \frac{g(a \mid b)}{g(b \mid a)}$
$A(b \rightarrow a)=1$

If If $\frac{f(b)}{f(a)} \times \frac{g(a \mid b)}{g(b \mid a)} \ge 1$:


$A(a \rightarrow b)=1$
$A(b \rightarrow a)=\frac{f(a)}{f(b)} \times \frac{g(b \mid a)}{g(a \mid b)}$

If $g$ is symmetric (like a gaussian), then
$A(a \rightarrow b)=MIN \left(1, \frac{f(b)}{f(a)}\right.$
$=MIN \left(1, \frac{\rho(b)}{p(a)}\right)$


Questions: Can we use something like accept-reject algorithm, or minimization or other methods?

In [42]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mp
from scipy.stats import multivariate_normal
import seaborn as sns
import pymc3 as pm
import arviz as az

plt.style.use('bmh')
colors = ['#348ABD', '#A60628', '#7A68A6', '#467821', '#D55E00', 
          '#CC79A7', '#56B4E9', '#009E73', '#F0E442', '#0072B2']
mp.rc('text', usetex=True)
RANDOM_SEED = 8927
np.random.seed(RANDOM_SEED)
az.style.use("arviz-darkgrid")

%matplotlib inline

In [7]:
#LOAD DATA
chi2_array_ALL_DATA_25k = np.load('/home/ali/Desktop/Pulled_Github_Repositories/NNPDF_Uncertainty/master_version/local/ALL_DATA_25k/chi2_array_ALL_DATA_25k.npy')
MVN_25k_MASTER = np.load('/home/ali/Desktop/Pulled_Github_Repositories/NNPDF_Uncertainty/master_version/samples/MVN_25k_MASTER.npy')
COV_MASTER= np.load('/home/ali/Desktop/Pulled_Github_Repositories/NNPDF_Uncertainty/master_version/samples/COV_MASTER.npy')
params_MASTER= np.load('/home/ali/Desktop/Pulled_Github_Repositories/NNPDF_Uncertainty/master_version/samples/params_MASTER.npy')

in our case, $f(x) = e^{-0.5\chi^2}$, $p(x) = L(\theta)$ the true likelihood of the parameters. Toy example

In [8]:
COV_MASTER

array([[ 4.45e-06,  3.44e-06,  2.25e-06,  2.27e-06, -5.33e-06,  1.28e-06,
         1.23e-06,  3.32e-05, -2.41e-06, -2.81e-06,  5.41e-06, -9.82e-07,
        -2.68e-05,  2.50e-05],
       [ 3.44e-06,  2.09e-04,  2.02e-06, -8.84e-06, -2.75e-04,  5.14e-05,
        -3.97e-06, -1.59e-03, -9.26e-05, -7.11e-04, -1.85e-04, -1.42e-05,
         4.77e-04, -1.93e-05],
       [ 2.25e-06,  2.02e-06,  2.33e-06,  1.28e-06, -3.87e-06,  1.07e-06,
         3.89e-07,  1.25e-05, -3.32e-06, -5.05e-06,  1.64e-05, -2.39e-07,
        -3.22e-05,  5.76e-06],
       [ 2.27e-06, -8.84e-06,  1.28e-06,  4.30e-04,  1.51e-05, -2.65e-06,
        -9.05e-06,  1.76e-03,  1.25e-03,  9.56e-05,  4.52e-04,  1.49e-06,
        -3.11e-04, -3.76e-04],
       [-5.33e-06, -2.75e-04, -3.87e-06,  1.51e-05,  3.76e-04, -5.76e-05,
         6.79e-06,  2.24e-03,  1.37e-04,  1.02e-03,  2.55e-04,  2.10e-05,
        -6.84e-04,  2.35e-05],
       [ 1.28e-06,  5.14e-05,  1.07e-06, -2.65e-06, -5.76e-05,  3.33e-05,
        -1.47e-06, -3.01e-04, -

In [9]:
MVN_25k_MASTER.shape

(25000, 14)

In [14]:
added = np.ones((1,14))
m = np.vstack((MVN_25k_MASTER, added))
m[-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [17]:
g = np.random.multivariate_normal(params_MASTER, COV_MASTER, 1)
g

array([[ 0.16139944,  0.0692491 , -0.12706395,  1.01073336, -0.05723465,
        -0.38216576,  0.80993938,  9.53986047,  4.83670809,  5.57800148,
         6.9421623 ,  4.81040456,  1.56024152,  9.99673951]])

In [26]:
float(g[:,2])

-0.12706395197763173

In [43]:
samples = np.empty((3,14))
s1=samples[1]
str(float(s1[2]))

'1.061671172211172e-153'

In [46]:
samples[0][0]

4.677353985613e-310